__Вопросы для повторения__:

<details>
<summary>заметка лектору в будущее на 2021 г.</summary>

Т.к. предыдущее занятие шло очень медленно, то вместо повторения будет доработка семинара прошлой лекции.
Актуальные вопросы для повторения можно подглядеть в лекции от 2019 г.

С другой стороны, стоит оценить, насколько такой подход к повторению лучше / хуже. Возможно, для материала "Классы. Продолжение" он оправдан.
    
</details>

<br />

##### Эхо прошлой лекции: разница между дополнением и переопределением в иерархии

**Пример 1**: классический ученический - поведение базового класса переопределено

```c++
// прямоугольник: высота и ширина
struct Rectangle
{
    double h;
    double w;
};

// ведь квадрат - частный случай прямоугольника?
struct Square : Rectangle
{
    Square(double side) : h(side), w(side) {}
};

// растягивать / сжимать прямоугольники без сохранения пропорций - это логично
void scale(Rectangle& rectangle, double scale_h, double scale_w)
{
    rectangle.h *= scale_h;
    rectangle.w *= scale_w;
}

// но что произойдёт в случае с квадратом?
Square s(5);
scale(s, 0.5, 0.7);  // ???
```

**Пример 2**: организация работы с сетью

*Вариант 1: наследник переопределяет поведение базового класса*

```c++
class Network {
public:
    virtual HttpResponse http_get(const std::string& url) {
        /* делаем честный запрос */ 
        /* замеряем время выполнения и записываем в |duration_ms| */
    }
    
    int latest_duration_ms() const {
        return duration_ms;
    }
    
private:
    int duration_ms = 0;
};

class UnavailableNetwork : public Network {
public:
    HttpResponse http_get(const std::string& url) override {
        return HttpResponse::Error(503);
    }
};  // код, работающий с Network, будет спрашивать latest_duration_ms(), и мы приплыли
```

*Вариант 2: наследник уточняет / дополняет поведение базового класса*

```c++
class INetwork {
public:
    virtual ~INetwork() = default;

    virtual HttpResponse http_get(const std::string& url) = 0;
    virtual int latest_duration_ms() const = 0;
};

class Network : public INetwork {
public:
    HttpResponse http_get(const std::string& url) override {
        /* ... как раньше - честный запрос с замером времени ... */
    }
    
    int latest_duration_ms() const override {
        return duration_ms;
    }
    
private:
    int duration_ms = 0;
};

class UnavailableNetwork : public INetwork {
public:
    HttpResponse http_get(const std::string& url) override {
        return HttpResponse::Error(503);
    }
    
    int latest_duration_ms() const override {
        return 0;
    }
};
```

<br />

##### Эхо прошлой лекции: виртуальные функции

Какая функция будет вызвана. Нарисовать что будет происходить в памяти

```c++
class A {
	~A() {}
	void f() {}
	virtual void g() {}
	void h() {}
};

class B : public A {
	virtual ~B() {}
	virtual void f() {}
	void g() {}
	void h() {}
};

class C : public B {
	~C() {}
	void f() {}
	void g() {}
	virtual void h() {}
};

int main()
{
	{
		C c;
		c.f(); c.g(); C.h();

		B b = c;
		b.f(); b.g(); b.h();

		A& ar = c;
		ar.f(); ar.g(); ar.h();
	}

	{
		C* cp = new C;
		A* ap = cp;
		ap->f(); ap->g(); ap->h();
		delete ap;
	}

	{
		C* cp = new C;
		B* bp = cp;
		delete bp;
	}

	return 0;
}
```

<br />

Какая функция будет вызвана?

```c++
class A {
public:
    A() {
        init();
    }

    ~A() {
        deinit();
    }

    virtual void init();
    virtual void deinit();
};

class B : public A {
public:
    B() {}
    ~B() {}

    void init() override { ... }
    void deinit() override { ... }
};

int main() {
    B b;
    return 0;
}
```

<br />

##### Эхо прошлой лекции: const-методы

Упражнение: какая функция будет вызвана

```c++
class A
{
public:
	void f() { std::cout << "f();"; }
	void f() const { std::cout << "f() const;" }
};


int main()
{
	A a;
	a.f();

	const A ca;
	ca.f();

	const A& car = a;
	car.f();

	A& ar = a;
	ar.f();

	A* const acp = &a;
	acp->f();

	const A* cap = &a;
	cap->f();

	const A a2 = a;
	a2.f();

	return 0;
}
```

<br />

##### Эхо прошлой лекции: class vs struct

* `class` - если есть инвариант
* `struct` - если поля независимы

```c++
struct Point
{
	double x;
	double y;
};

struct Color
{
	char r;
	char g;
	char b;
};

// ???
struct Size
{
	double w;
	double h;
};

struct Rectangle
{
	Point origin;
	Size size;
};

class JuiceBottle
{
private:
	double max_volume_;
	double cur_volume_;
};

class String
{
private:
	char *s_;
	size_t len_;
	size_t capacity_;
};
```

<br />

##### Эхо прошлой лекции: Принципы дизайна класса

* RAII
* Есть инвариант - `class`, поля независимы - `struct`
* Если есть подклассы - делаем `virtual` деструктор
* Правило 6
    * `=default` где достаточно автогенерённого
    * `=delete` где нужно запретить

* `noexcept` d-tor, move operators (подробнее на лекции про исключения)
* `const` на все методы, которые не меняют состояния класса (значения полей)
* `override` на переопределённые `virtual` методы
* рекомендовано избегать `protected`-полей
* Какой функционал делать методом, а какой - свободной фунцкией? Всопним ADL. Благодаря ему есть философский принцип, что функции, принимающие первым аргументом объект класса (или ссылку), являются частью интерфейса класса:

    ```c++
    struct Vector3D
    {
        double x, y, z;
    };

    // философски часть интерфейса класса Vector3D
    double length(const Vector3D v);
    ```

    Поэтому принцип рационального минмимума в методах:
      * struct Vector3D - всё в свободные функции (normalize, length, rotate ...))
      * vector<int> (pointer, size, capacity: size() - member; at() - member; sort() - function; empty() - ???; middle element - function; max() - function, mean() - function)
      * cache (add, get, del; clear? - в метод, т.к. эффективнее реализовать, обладая доступом до полей)

    Нужна __причина__, чтобы что-нибудь сделать методом:
        * требуется доступ до приватных полей и нельзя реализовать иначе (vector.size(), vector.at(), vector.push_back())
        * реализация через приватные поля проще и быстрее (cache.clear())
        * лучше вписывается в сщуествующие методы класса (vector.empty())
        * что-нибудь ещё?

* Иногда нужно менять поля в `const`-методах, и это не влияет на состояние класса в умозрительном смысле. Используйте `mutable`:
    * Кеширование: бинарное дерево, которое запоминает последний запрос
    * `mutex`
    * отложенная инициализация

* SRP - Single Responsibility Principle - Принцип единственной ответственности - класс должен быть экспертом в одной и только одной области. Такие классы проще отлаживать и тестировать. Всегда понятно, ошибка в рамках его области ответственности или нет.
* Liskov Substitution Principle - Принцип подстановки Барбары Лисков - наследник класса всегда можно передать в алгоритм вместо базового, при этом поведение алгоритма не изменится. Другими словами: наследники расширяют поведение базовых классов, но не изменяют его.

<br />

### Лекция 5. Шаблоны

<br />

##### Какая идея стоит за шаблонами

Ранее мы познакомились с возможностью перегрузки функций. Давайте вспомним её на примере swap:

```c++
// поменять местами два int
void my_swap(int& a, int& b)
{
    int tmp = a;
    a = b;
    b = tmp;    
}

// поменять местами два short
void my_swap(short& a, short& b)
{
    short tmp = a;
    a = b;
    b = tmp;
}

// поменять местами два float
void my_swap(float& a, float& b)
{
    float tmp = a;
    a = b;
    b = tmp;
}

...
```

Вечер начинает быть томным ...

Для решения проблем написания одинакового кода придуманы шаблоны:

```c++
// напишем шаблон - как должна выглядеть функция
template<typename T>
void my_swap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}
```

Применение шаблона:

```c++
int a = 3, b = 5;

// вызов my_swap(int&, int&), тип T указывается программистом явно
my_swap<int>(a, b);

// вызов my_swap(int&, int&), тип T выводится компилятором автоматически
my_swap(a, b); 


float x = 3.f, y = 5.f;
my_swap(x, y);
my_swap<float>(x, y);
```

Важное свойство шаблонов по сравнению с перегрузкой функций: шаблон компилируется только тогда, когда он вызывается. И компилируется только для тех типов, для которых он вызывается.

_но в каждом cpp-файле шаблон компилируется снова и снова_

Показать пример на godbolt.org, позакомментировать функции, продемонстрировать разницу в выхлопе компилятора.

```c++
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}
```

Особенности шаблонов по сравнению с перегруженными функциями:
* компилируется только то, что инстанциируется в коде
* компилируется столько раз, в скольки единицах трансляции инстанциируется:
    * можно в одном cpp-файле 10 раз позвать myswap(int&, int&) - эта функция скомпилируется единожды
    * можно в 10 cpp-файлах один раз позвать myswap(int&, int&) - эта функция скомпилируется 10 раз
* накладные расходы во время компиляции на кодогенерацию при истанциации
* позволяет компилятору агрессивнее оптимизировать. Раскомментировать `__attribute__((noinline))` из примера и показать какой код сгенерирует компилятор. Объяснить, почему.
* позволяет нарушать ODR

Коротко:

* (+) меньше кода
* (+) быстрее
* (-) дольше компилируется
* (-) сложнее писать

__Вопросы__:
* Где поместить шаблонную функцию, которую нужно использовать в разных cpp-файлах?
* Где поместить её реализацию?
* Может ли шаблонная функция содержать некомпилирующийся код?

<br />

##### Специализация

Перегрузка функций позволяла сделать `myswap` у `std::string` более эффективно, без лишнего копирования памяти:

```c++
void myswap(int& a, int& b) { ... }
void myswap(short& a, short& b) { ... }

void myswap(std::string& a, std::string& b)
{
    a.swap(b);
}
```

Шаблоны тоже позволяют специализировать поведение функций, если наложить на шаблонный параметр ограничение, например:

(закинуть этот код на godbolt, показать во что компилируется программа)

```c++
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

template<>
void __attribute__ ((noinline)) myswap<std::string>(std::string& a, std::string& b)
{
    a.swap(b);
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}
```

Во-первых, шаблон может иметь несколько параметров, а во-вторых, параметры не обязаны быть типами. Они могут быть, например, целыми числами:

```c++
template<int N, typename T>
T add_value(T x)
{
    return x + N;
}

int a = add_value<5, int>(100);
int a = add_value<5>(100);

// 1. шаблон специализирован программистом частично, тип Т компилятор определит сам
// 2. параметром шаблона выступает целое число.
```

__Вопрос__: Какую информацию здесь компилятор использует, чтобы вывести тип `T`?

<br />

##### Шаблонные классы

Аналогично функциям, классы тоже могут быть шаблонными:

Пример структуры:

```c++
// N-мерный вектор из курса линейной алгебры типа T
template<typename T, int N>
struct VectorN
{
    T data[N];
};

// в качестве примера запишем операции сложения и умножения для таких векторов

template<typename T, int N>
VectorN<T, N> operator +(const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    VectorN<T, N> rv;
    for (int i = 0; i < N; ++i)
        rv.data[i] = l.data[i] + r.data[i];
    return rv;    
}

template<typename T, int N>
T operator * (const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    T rv = 0;
    for (int i = 0; i < N; ++i)
        rv += l.data[i] * r.data[i];
    return rv;
}
```

Пример класса:

```c++
template<typename T, int N>
class RoundRobinQueue
{
private:
    T arr[N];
    int begin_ix = 0;
    int end_ix = 0;

public:
    void push(const T& item)
    {
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    T pop()
    {
        T item = arr[begin_ix];
        begin_ix = next_ix(begin_ix);
        return item;
    }

    bool empty() const
    {
        return begin_ix == end_ix;
    }
    
    bool full() const
    {
        return next_ix(end_ix) == begin_ix;
    }
    
private:
    static int next_ix(const int ix)
    {
        return (ix + 1) % N;
    }
};
```

Его использование:

```c++
RoundRobinQueue<int, 100> queue;
queue.push(1);
queue.push(2);
queue.push(3);
queue.pop();  // 1
queue.pop();  // 2
queue.pop();  // 3
```

__Вопрос__: Как обходить вопрос, что программист может позвать ещё один `queue.pop` ?

А ещё класс может иметь шаблонный метод, почему бы и нет.

```c++
#include <array>
#include <string>

template<typename T, int N>
class RoundRobinQueue
{
private:
    std::array<T, N> arr;
    int begin_ix = 0;
    int end_ix = 0;

public:
    static int next_ix(const int ix)
    {
        return (ix + 1) % N;
    }

    template<typename U>
    void push(const U& item)
    {
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    T pop()
    {
        T item = arr[begin_ix];
        begin_ix = next_ix(begin_ix);
        return item;
    }

    bool empty() const
    {
        return begin_ix == end_ix;
    }

    bool full() const
    {
        return next_ix(end_ix) == begin_ix;
    }
};

std::string f()
{
    RoundRobinQueue<std::string, 100> queue;
    queue.push(std::string("run"));
    queue.push("Forest");
    queue.push("run");
    return queue.pop();
}
```

Закинуть пример на godbolt, показать, что генерируется 3 метода `push`, не забыть убрать оптимизации

Шаблонный конструктор - пожалуйста! Это ведь тоже метод

Шаблонный деструктор... пожалуй, нет, хватит.

<br />

##### Частичная специализация шаблонов (для классов)

Например, можно прописать alias:

```c++
template<int N>
using RoundRobinStringsQueue = RoundRobinQueue<std::string, N>;


RoundRobinStringsQueue<100> queue;
```

А можно и изменить поведение класса

```c++
template<int N>
class RoundRobinQueue<std::string, N>
{
private:
    std::string arr[N];  // !!! here
    int begin_ix = 0;
    int end_ix = 0;

public:
    ...
    
    template<typename U>
    void push(const U& item)
    {
        std::cout << "push!";  // !!! here
        arr[end_ix] = item;
        end_ix = next_ix(end_ix);
    }
    
    ...
};
```

Такие сложности могут пригодиться для compile-time полиморфизма. С его классическим применением - type traits-ами - познакомимся позже.

<br />

##### just for fun: compile-time факториал

Теперь мы разбираемся в шаблонах достаточно чтобы посчитать факториал во время компиляции на шаблонах (разобрать пример, показать результат в godbolt).

Примечание: C++ значительно эволюционировал, и больше во время компиляции таким образом вычисления не проводят. Пример исключительно ученический. Compile-time вычисления будут рассмотрены в курсе далее.

```c++
template<unsigned N>
struct Factorial
{
    static const int value = N * Factorial<N - 1>::value;
};

template<>
struct Factorial<0>
{
    static const int value = 1;
};

int main()
{
    return Factorial<10>::value;
}
```

__Вопросы__:
* Что делает следующий пример?

```c++
#include <cstdio>

template<unsigned N>
struct f
{
    static const int value = f<N-1>::value + f<N-2>::value;
};

template<>
struct f<0>
{
    static const int value = 0;
};

template<>
struct f<1>
{
    static const int value = 1;
};

int main()
{
    printf("%i\n", f<45>::value);
}
```

<br />

Во второй части курса про шаблоны:
* SFINAE
* variadic templates
* type traits
* tag dispatching

<br />

**Замечания после лекции**:

1. Удалить упоминание принципа Барбары Лисков. Т.к. подобрать хороший пример не получается, то, видимо, лектор сам недостаточно хорошо понимает этот принцип.
2. Материала лекции по шаблонам мало. Даже с эхом от предыдущей лекции на занятие не хватило, осталось 30+ минут. Нужно либо дополнять (выдача домашки / разбор ДЗ / зайти поглубже в шаблоны) либо вводить параллельную тему.
3. Текст про компиляцию и разделение на файлы нужно иллюстрировать примерами. На слух воспринимается ужасно.
4. Из семинара убрать дубликат параграфа про виртуальный деструктор
5. Нужен яркий пример для параграфа member types. Сейчас пример искуственный.